<a href="https://colab.research.google.com/github/ufrpe-mineracao-textos/projeto-de-mineracao-20192-ner-analise-juridica/blob/master/analise_andamentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [0]:
import pandas as pd
import os
import re
import spacy
from spacy import displacy


import pickle
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier

In [0]:
os.system("python -m spacy download pt")

0

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My\ Drive/Academico/8\º\ Período/Mineração\ de\ Texto/Projeto\ de\ MT

/content/drive/My Drive/Academico/8º Período/Mineração de Texto/Projeto de MT


In [0]:
!ls

 analise_andamentos.ipynb   Model		 sentence_classification.ipynb
 Database		    ner_juridico.ipynb
'Ficha projeto.gsheet'	   'Related Works'


## Import da base

In [0]:
base_ouvidoria = pd.read_json('Database/processos_ouvidoria.json')
base_sequencial = pd.read_json('Database/processos_sequencial.json')

In [0]:
base = pd.concat([base_ouvidoria, base_sequencial],ignore_index=True)

In [0]:
base = base.drop_duplicates(subset='numero')

base.head()

,numero,juizo,classe_CNJ,eletronico,comarca,instancia,estado,andamentos_qtd,assuntos,andamentos
0,1774826420128169984,Segunda Vara de Violência Doméstica e Familiar...,Inquérito Policial,False,Recife,1GRAU,PE,4,[Crimes Previstos no Estatuto do Idoso],"[{'complemento': 'Central de Inquéritos', 'dat..."
1,1736234020128169984,Segunda Vara de Violência Doméstica e Familiar...,Inquérito Policial,False,Recife,1GRAU,PE,22,[Crimes Previstos no Estatuto do Idoso],"[{'complemento': 'Central de Inquéritos', 'dat..."
2,1736087120128169984,Terceira Vara de Violência Doméstica e Familia...,Inquérito Policial,False,Recife,1GRAU,PE,7,[Ameaça],[{'complemento': 'Terceira Vara de Violência D...
3,1736051920128169984,Primeira Vara de Violência Doméstica e Familia...,Inquérito Policial,False,Recife,1GRAU,PE,94,"[Injúria, Ameaça, Constrangimento ilegal]","[{'complemento': 'Petição', 'data': '07/06/201..."
4,1736034920128169984,Primeira Vara de Violência Doméstica e Familia...,Inquérito Policial,False,Recife,1GRAU,PE,8,[Crimes Previstos no Estatuto do Idoso],"[{'complemento': 'Central de Inquéritos', 'dat..."


In [0]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13300 entries, 0 to 19217
Data columns (total 10 columns):
numero            13300 non-null int64
juizo             13300 non-null object
classe_CNJ        13300 non-null object
eletronico        13300 non-null bool
comarca           13300 non-null object
instancia         13300 non-null object
estado            13300 non-null object
andamentos_qtd    13300 non-null int64
assuntos          13300 non-null object
andamentos        13300 non-null object
dtypes: bool(1), int64(2), object(7)
memory usage: 1.0+ MB


## Leitura dos andamentos

### OBS

- Não podemos remover a data!! precisamos dela para dizer quanto tempo durou o processo

In [0]:
def remove_region(ent, text):
    old = text[ent.start_char:ent.end_char]
    new = []

    for word in list(old):
        if word != ' ':
            new.append('x')
        else:
            new.append(' ')

    new = ''.join(new)

    return text.replace(old, new)\

In [0]:
with open('Model/base', 'rb') as fp:
    sentences_train, labels_train = pickle.load(fp)

tfidf = TfidfVectorizer(smooth_idf=True, norm='l1', max_features=1000)
x_train_tfidf = tfidf.fit_transform(sentences_train)

# clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf = MLPClassifier(activation='relu', alpha=1e-05,
                    batch_size='auto', beta_1=0.9,
                    beta_2=0.999, early_stopping=False,
                    epsilon=1e-08, hidden_layer_sizes=(5, 2),
                    learning_rate='constant',
                    learning_rate_init=0.001, max_iter=200,
                    momentum=0.9, n_iter_no_change=10,
                    nesterovs_momentum=True, power_t=0.5,
                    random_state=1, shuffle=True, solver='lbfgs',
                    tol=0.0001, validation_fraction=0.1,
                    verbose=False, warm_start=False)
clf.fit(x_train_tfidf, labels_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
def detect_sentences(progress, nlp):
    decrees = []
    doc = nlp(progress)
    doc_test = [str(sent) for sent in doc.sents]
    if len(doc_test) > 0:
        doc_tfidf = tfidf.transform(doc_test)
        predict = clf.predict(doc_tfidf)
        for idx2, label in enumerate(predict):
            if label:
                decrees.append(doc_test[idx2])
    return decrees

In [0]:
len(base['andamentos'])

13300

In [0]:
matchs = ['\d{1,2}\/\d{1,2}\/\d{2,4}', '\d{7}-\d{2}\.\d{4}\.\d{1}\.\d{2}\.\d{4}',
          '\d{1,2}\.\d{1,2}\.\d{2,4}', '\d+ de \w+ de \d+', '\d{2,4}:\d{2,4}:\d{2,4}']
my_model = spacy.load('pt')


for i, andamentos in enumerate(base['andamentos']):
    print('Indice {}'.format(i), end=' ')
    new_andamentos = []
    
    for andamento in andamentos:
        andamento_aux = {}
        
        for key in andamento.keys():
            text = andamento[key]
            
            if key != 'data' and key != 'decretos':                
                document = my_model(text)

                for ent in document.ents:
                    if ent.label_ == 'PER' and ent.text.isupper():
                        text = remove_region(ent, text)
                
                for regex in matchs:
                    text = re.sub(regex, 'xxxxxxxx', text)
                
                if key == 'texto':
                    decretos = detect_sentences(text, my_model)
                    andamento_aux['decretos'] = decretos
        
            andamento_aux[key] = text
        
        new_andamentos.append(andamento_aux)
    
    base['andamentos'][i] = new_andamentos
    new_andamentos = []
    if i == 100: break

Indice 0 Indice 1 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Indice 2 Indice 3 Indice 4 Indice 5 Indice 6 Indice 7 Indice 8 Indice 9 Indice 10 Indice 11 Indice 12 Indice 13 Indice 14 Indice 15 Indice 16 Indice 17 Indice 18 Indice 19 Indice 20 Indice 21 Indice 22 Indice 23 Indice 24 Indice 25 Indice 26 Indice 27 Indice 28 Indice 29 Indice 30 Indice 31 Indice 32 Indice 33 Indice 34 Indice 35 Indice 36 Indice 37 Indice 38 Indice 39 Indice 40 Indice 41 Indice 42 Indice 43 Indice 44 Indice 45 Indice 46 Indice 47 Indice 48 Indice 49 Indice 50 Indice 51 Indice 52 Indice 53 Indice 54 Indice 55 Indice 56 Indice 57 Indice 58 Indice 59 Indice 60 Indice 61 Indice 62 Indice 63 Indice 64 Indice 65 Indice 66 Indice 67 Indice 68 Indice 69 Indice 70 Indice 71 Indice 72 Indice 73 Indice 74 Indice 75 Indice 76 Indice 77 Indice 78 Indice 79 Indice 80 Indice 81 Indice 82 Indice 83 Indice 84 Indice 85 Indice 86 Indice 87 Indice 88 Indice 89 Indice 90 Indice 91 Indice 92 Indice 93 Indice 94 Indice 95 Indice 96 Indice 97 Indice 98 Indice 99 Indice 100 

In [0]:
base.to_json('Database/andamentos_limpos.json')

FileNotFoundError: ignored

In [0]:
base['andamentos'][2]

[{'complemento': 'Terceira Vara de Violência Doméstica e Familiar contra a Mulher na Comarca da Capital',
  'data': '18/01/2016 12:02:00',
  'decretos': [],
  'texto': '',
  'titulo': 'Redistribuído por Prazo 7 - 1ª D - MAR/JUN/SET/DEZ'},
 {'complemento': 'Ciente nos autos',
  'data': '08/01/2016 14:54:00',
  'decretos': [],
  'texto': '',
  'titulo': 'Recebidos os autos Ciente nos autos'},
 {'complemento': 'Ministério Público',
  'data': '28/10/2015 14:39:00',
  'decretos': [],
  'texto': '',
  'titulo': 'Autos entregues em carga ao Ministério Público'},
 {'complemento': 'DJe NÂº: 49/2015 Data PublicaÃ§Ã£o: xxxxxxxx',
  'data': '19/02/2015 17:35:00',
  'decretos': ['Acusado:', 'xxxxx xxxx DE OLIVEIRA\r\nVítima:'],
  'texto': '2ª Vara de Violência Doméstica e Familiar Contra a Mulher\r\n\r\nAcusado: xxxxx xxxx DE OLIVEIRA\r\nVítima: ALCIONE MARIA MELO DE OLIVEIRA\r\nProcesso nº xxxxxxxx\r\n\r\n     SENTENÇA\r\n\r\n             Vistos etc...\r\n\r\n       xxxxx xxxx DE OLIVEIRA, devidam